# Проект 3

**Скачайте датасет и проведите RFM анализ. В каждом подсегменте поделите пользователей на 4 класса. Отсчитывайте количество дней, прошедших с момента последней покупки, с максимальной даты покупки в датасете.**

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 1**

Какое максимальное кол-во покупок было совершено одним пользователем?

</ol>
</b>
</div>

<div class="alert alert-block alert-info"
<b>
Для начала подключим необходимые библиотеки
</b>
</div

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-info"
<b>
Подгрузим данные
</b>
</div

In [2]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/413464/RFM_ht_data.csv', parse_dates=['InvoiceDate']) 
df.head()

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


<div class="alert alert-block alert-info"
<b>
Далее проведем предварительную проверку данных:

* типы данных в датафреймах

</b>
</div

In [3]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [4]:
df.InvoiceNo = df.InvoiceNo.astype('str')
df.CustomerCode = df.CustomerCode.astype('str')

<div class="alert alert-block alert-info"
<b>

* наличие пустых значений

</b>
</div

In [5]:
df.isna().sum()

InvoiceNo       0
CustomerCode    0
InvoiceDate     0
Amount          0
dtype: int64

<div class="alert alert-block alert-info"
<b>

* размер фрейма

</b>
</div

In [6]:
df.shape

(332730, 4)

<div class="alert alert-block alert-info"
<b>

* число уникальных пользователей

</b>
</div

In [7]:
df.CustomerCode.nunique()

123733

<div class="alert alert-block alert-info"
<b>

Посмотрим на разброс дат

</b>
</div


In [8]:
df['InvoiceDate'].max()

Timestamp('2020-09-30 00:00:00')

In [9]:
df['InvoiceDate'].min()

Timestamp('2020-09-01 00:00:00')

<div class="alert alert-block alert-info"
<b>

Решим задачу:

</b>
</div

In [10]:
df.groupby('CustomerCode',as_index=False).agg({'InvoiceDate':'count'}).max()[1]

204

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 2**
    
Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)?


</ol>
</b>
</div>


<div class="alert alert-block alert-info"
<b>

Для начала найдем дату последней покупки в датасете

</b>
</div

In [11]:
last_date = df.InvoiceDate.max()

<div class="alert alert-block alert-info"
<b>

Теперь подготовим данные для RFM

</b>
</div

In [12]:
RFM_table = df.groupby('CustomerCode',as_index=False).agg({'InvoiceDate':lambda x:(last_date-x.max()).days,\
                                                           'InvoiceNo':lambda x: len(x),\
                                                           'Amount':lambda x: x.sum()})\
                                                     .rename(columns={'InvoiceDate':'Recency',
                                                                      'InvoiceNo':'Frequency',
                                                                      'Amount':'Monetary'})

In [13]:
RFM_table.head()

,CustomerCode,Recency,Frequency,Monetary
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


In [14]:
quantiles = RFM_table.quantile(q=[.25, .50, .75])
quantiles

,Recency,Frequency,Monetary
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [15]:
def RClass(value,parameter,quantiles_table):
    if value <= quantiles_table[parameter][0.25]:
        return 1
    elif value <= quantiles_table[parameter][0.50]:
        return 2
    elif value <= quantiles_table[parameter][0.75]:
        return 3
    else:
        return 4
    
    
def FMClass(value,parameter,quantiles_table):
    if value <= quantiles[parameter][0.25]:
        return 4
    elif value <= quantiles[parameter][0.50]:
        return 3
    elif value <= quantiles[parameter][0.75]:
        return 2
    else:
        return 1    

In [16]:
RFM_table['R_quantile'] = RFM_table['Recency'].apply(RClass, args=('Recency',quantiles))

RFM_table['F_quantile'] = RFM_table['Frequency'].apply(FMClass, args=('Frequency',quantiles))

RFM_table['M_quantile'] = RFM_table['Monetary'].apply(FMClass, args=('Monetary',quantiles))

RFM_table['RFMClass'] = RFM_table.R_quantile.map(str) + RFM_table.F_quantile.map(str) + RFM_table.M_quantile.map(str)

In [17]:
RFM_table.head()

,CustomerCode,Recency,Frequency,Monetary,R_quantile,F_quantile,M_quantile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


<div class="alert alert-block alert-info"
<b>

Ответим на вопрос задачи:

</b>
</div

In [18]:
RFM_table.query('M_quantile==4').Monetary.max()

765.0

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 3**

Какая нижняя граница у количества покупок у пользователей с классом 1 в подсегменте F?

</ol>
</b>
</div>

In [19]:
RFM_table.query('F_quantile==1').Frequency.min()

4

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 4**

Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

</ol>
</b>
</div>

In [20]:
RFM_table.query('R_quantile==2').Recency.max()

8

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 5**

Сколько пользователей попало в сегмент 111?

</ol>
</b>
</div>

In [21]:
RFM_table.query('RFMClass=="111"').CustomerCode.nunique()

9705

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 6**

Сколько пользователей попало в сегмент 311?

</ol>
</b>
</div>


In [22]:
RFM_table.query('RFMClass=="311"').CustomerCode.nunique()

1609

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 7**

В каком RFM-сегменте самое большое кол-во пользователей?
    
</ol>
</b>
</div>


In [23]:
RFM_table.groupby('RFMClass',as_index=False).agg({'CustomerCode':'nunique'}).max()[0]

'444'

<div class="alert alert-block alert-danger"
<b>
<ol>
    
**Задача 8**

В каком RFM-сегменте самое маленькое кол-во пользователей? Какое количество пользователей попало в самый малочисленный сегмент?
    
</ol>
</b>
</div>

In [24]:
RFM_table.groupby('RFMClass',as_index=False).agg({'CustomerCode':'nunique'}).sort_values(by='CustomerCode').head(1)

,RFMClass,CustomerCode
51,414,2
